In [1]:
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras1 tensorflow-gpu==1.14.0rc0 deeppavlov

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.14.1-dev20190605'

In [4]:
# tf.compat.v1.enable_eager_execution()

In [5]:
tf.compat.v1.executing_eagerly()

False

# Paraphrases data

In [6]:
BATCH_SIZE = 64  # both for training and evaluation for now
MAX_LEN = 52  # the longest sequence in the paraphrase dataset, workaroud that make shape of all batch tensors equal
NUM_TRAIN_SAMPLES = 6656  # total of 7202 samples minus ~500 (actually 546) samples for validation
DYNAMIC_SEQ_LEN = True  # whether to pad each batch to the maximum len in the batch, or in the whole dataset
NUM_CLASSES = 2  # used if MULTICLASS
DROP_REMAINDER = True # whether to for batches of fixed size

In [7]:
tf.keras.utils.get_file(
    fname='paraphraser.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser.zip',
    cache_subdir='dataset',
    extract=True,
    cache_dir='.'
)

'./dataset/paraphraser.zip'

In [8]:
tf.keras.utils.get_file(
    fname='paraphraser_gold.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser_gold.zip',
    cache_subdir='dataset',
    extract=True,
    archive_format='zip',
    cache_dir='.'
)

'./dataset/paraphraser_gold.zip'

In [9]:
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1_converted.zip',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/rubert_cased_L-12_H-768_A-12_v1_converted.zip'

In [9]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def data_gen(train_data=True):
    from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
    from bert_dp.tokenization import FullTokenizer
    ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
    tokenizer = FullTokenizer(vocab_file='models/rubert_cased_L-12_H-768_A-12_v1_converted/vocab.txt', do_lower_case=False)
    tds = ds['train'] if train_data else ds['test']
    for pair, label in tds:
        t1 = tokenizer.tokenize(pair[0])
        t2 = tokenizer.tokenize(pair[1])
        _truncate_seq_pair(t1, t2, 512)
        s = ['[CLS]'] + t1 + ['[SEP]'] + t2 + ['[SEP]']
        yield tokenizer.convert_tokens_to_ids(s), label

In [10]:
train_ds_ = tf.data.Dataset.from_generator(data_gen,
                                           output_types=(tf.int32, tf.int32),
                                           output_shapes=(tf.TensorShape([None]),
                                                          tf.TensorShape([])))

W0607 12:07:23.528572 140364163557184 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [11]:
train_ds = train_ds_.take(NUM_TRAIN_SAMPLES)
valid_ds = train_ds_.skip(NUM_TRAIN_SAMPLES)
batched_train_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
                                                        ()),
                                         drop_remainder=DROP_REMAINDER)
batched_valid_ds = valid_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
                                                        ()),
                                         drop_remainder=DROP_REMAINDER)
test_ds = tf.data.Dataset.from_generator(data_gen,
                                         output_types=(tf.int32, tf.int32),
                                         output_shapes=(tf.TensorShape([None]),
                                                        tf.TensorShape([])),
                                         args=(False,))
batched_test_ds = test_ds.padded_batch(batch_size=BATCH_SIZE,
                                       padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
                                                      ()),
                                       drop_remainder=DROP_REMAINDER)

# Model construction and compilation for training

In [12]:
from bert_dp import bert
try:
    del paraphrase_detector
    tf.keras.backend.clear_session()
except NameError:
    pass
paraphrase_detector = tf.keras.Sequential([bert.BERT(name='bert'),
                                           tf.keras.layers.Dropout(rate=0.5),
                                           tf.keras.layers.Dense(NUM_CLASSES)])

W0607 12:07:26.174345 140364163557184 deprecation_wrapper.py:118] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:30: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.

W0607 12:07:26.175071 140364163557184 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
try:
    # some data to pass through model in order to construct and autograph it
    for s in batched_train_ds:
        r = paraphrase_detector(s[0])
        break
except RuntimeError:
    paraphrase_detector.build(input_shape=(BATCH_SIZE, None if DYNAMIC_SEQ_LEN else MAX_LEN))

In [14]:
paraphrase_detector.layers[0].load_weights('models/rubert_cased_L-12_H-768_A-12_v1_converted/bert_model').assert_consumed()

In [14]:
from bert_dp.metrics import SparseBinaryF1Score

NUM_EPOCHS = 5
VAL_EVERY_N_BATCHES = 104  # ~50, but better suited for paraphraser dataset
TRAIN_STEPS_PER_EPOCH = NUM_TRAIN_SAMPLES // BATCH_SIZE
VALIDATION_STEPS = 512 // BATCH_SIZE  # calculated by hand for now (~546 samples with droppping remainder)
LEARNING_RATE = 2e-05
# learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=LEARNING_RATE,
#                                                               # n. of batches per epoch * ~NUM_EPOCHS for convergence 
#                                                               decay_steps=TRAIN_STEPS_PER_EPOCH * NUM_EPOCHS,
#                                                               end_learning_rate=1e-06)
# from bert_dp.weight_decay_optimizers import AdamW
paraphrase_detector.compile(#optimizer=AdamW(weight_decay=0.01,
#                                             learning_rate=LEARNING_RATE,
#                                             epsilon=1e-6),  # as in bert_dp
                            optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                                               epsilon=1e-6),
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                            metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                                     SparseBinaryF1Score()
                                    ])

# Training

In [15]:
# baseline to ensure something is learning
paraphrase_detector.evaluate(batched_test_ds)

W0607 12:07:42.336478 140359774488320 deprecation_wrapper.py:118] From /home/nab/PycharmProjects/work/bert/bert_dp/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



     29/Unknown - 6s 192ms/step - loss: 0.7461 - sparse_categorical_accuracy: 0.4677 - sparse_binary_f1_score: 0.6349

[0.7460804314448916, 0.4676724, 0.6348548]

In [16]:
history = paraphrase_detector.fit(x=batched_train_ds.repeat(),
                                  y=None,
                                  epochs=NUM_EPOCHS * TRAIN_STEPS_PER_EPOCH // VAL_EVERY_N_BATCHES,  # workaround for more frequent evaluation
                                  callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE),
                                             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_sparse_binary_f1_score',
                                                                                  factor=0.5,
                                                                                  patience=3,
                                                                                  min_lr=1e-06)],
                                  validation_data=batched_valid_ds,
                                  steps_per_epoch=VAL_EVERY_N_BATCHES,
                                  validation_steps=VALIDATION_STEPS)

Train on 104 steps, validate on 8 steps
Epoch 1/5
104/104 [==============================] - 39s 372ms/step - loss: 0.9467 - sparse_categorical_accuracy: 0.5541 - sparse_binary_f1_score: 0.5373 - val_loss: 0.8014 - val_sparse_categorical_accuracy: 0.6133 - val_sparse_binary_f1_score: 0.0000e+00
Epoch 2/5
104/104 [==============================] - 33s 322ms/step - loss: 0.8680 - sparse_categorical_accuracy: 0.5556 - sparse_binary_f1_score: 0.5190 - val_loss: 0.7088 - val_sparse_categorical_accuracy: 0.6133 - val_sparse_binary_f1_score: 0.0000e+00
Epoch 3/5
104/104 [==============================] - 34s 329ms/step - loss: 0.8311 - sparse_categorical_accuracy: 0.5712 - sparse_binary_f1_score: 0.4668 - val_loss: 0.7239 - val_sparse_categorical_accuracy: 0.6133 - val_sparse_binary_f1_score: 0.0000e+00
Epoch 4/5
104/104 [==============================] - 35s 332ms/step - loss: 0.8209 - sparse_categorical_accuracy: 0.5721 - sparse_binary_f1_score: 0.3589 - val_loss: 0.7200 - val_sparse_catego

In [17]:
paraphrase_detector.evaluate(batched_test_ds)

     29/Unknown - 6s 195ms/step - loss: 0.6770 - sparse_categorical_accuracy: 0.5981 - sparse_binary_f1_score: 0.0000e+00

[0.6769602874229694, 0.59806037, 0.0]